# 🚀 Fairness Framework - Quick Demo

**Duration**: 5-10 minutes

This notebook demonstrates the basic usage of the Fairness Framework for detecting bias in machine learning models.

## What you'll learn:
1. How to load data
2. How to detect bias automatically
3. How to interpret fairness metrics
4. How to visualize results

## 1. Setup and Imports

In [ ]:
# Add parent directory to path
import sys
sys.path.insert(0, '../..')

# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Fairness framework
from src.fairness_detector import FairnessDetector
from src.metrics import compute_all_metrics
from src.visualization import plot_fairness_report, plot_group_comparison

# Display settings
pd.set_option('display.max_columns', None)
%matplotlib inline

print("✓ Imports successful!")

## 2. Create Synthetic Data (for demonstration)

Let's create a simple dataset with intentional bias for demonstration purposes.

In [ ]:
np.random.seed(42)

# Create synthetic dataset with bias
n_samples = 1000

# Features
feature_1 = np.random.randn(n_samples)
feature_2 = np.random.randn(n_samples)

# Sensitive attribute (binary: Group A vs Group B)
sensitive_attr = np.random.choice(['Group_A', 'Group_B'], size=n_samples)

# Create target with intentional bias
# Group A has higher probability of positive outcome
target = []
for i in range(n_samples):
    if sensitive_attr[i] == 'Group_A':
        prob = 0.7  # 70% positive rate for Group A
    else:
        prob = 0.4  # 40% positive rate for Group B (BIAS!)
    
    # Add some randomness based on features
    prob += 0.1 * feature_1[i] + 0.05 * feature_2[i]
    prob = np.clip(prob, 0, 1)
    
    target.append(1 if np.random.rand() < prob else 0)

# Create DataFrame
df = pd.DataFrame({
    'feature_1': feature_1,
    'feature_2': feature_2,
    'sensitive_attr': sensitive_attr,
    'target': target
})

print(f"Dataset created: {df.shape[0]} samples, {df.shape[1]} columns")
print(f"\nTarget distribution by group:")
print(df.groupby('sensitive_attr')['target'].agg(['count', 'mean', 'sum']))

## 3. Initialize Fairness Detector

In [ ]:
# Create detector instance
detector = FairnessDetector(threshold=0.1, verbose=True)

# Set configuration
detector.set_sensitive_attributes(['sensitive_attr'])
detector.set_target('target')

print("\n✓ Detector configured!")

## 4. Detect Bias

In [ ]:
# Run bias detection
results = detector.detect_bias(df)

# Print summary
print(results.summary())

## 5. Compute All Fairness Metrics

In [ ]:
# Compute all available metrics
all_metrics = compute_all_metrics(
    y_true=df['target'],
    y_pred=df['target'],  # Using actual labels for simplicity
    sensitive_attr=df['sensitive_attr']
)

print("All Fairness Metrics:")
print("=" * 50)
for metric_name, value in all_metrics.items():
    status = "✓ FAIR" if abs(value) <= 0.1 else "✗ BIASED"
    print(f"{metric_name:25s}: {value:7.4f}  [{status}]")

## 6. Visualize Results

In [ ]:
# Plot comprehensive fairness report
plot_fairness_report(results)

In [ ]:
# Plot group comparison
plot_group_comparison(
    y_pred=df['target'],
    sensitive_attr=df['sensitive_attr'],
    group_names=['Group A', 'Group B']
)

## 7. Interpretation

### What do these metrics mean?

- **Demographic Parity**: Difference in positive prediction rates between groups
  - Value close to 0 = Fair
  - |value| > 0.1 = Potential bias

- **Equalized Odds**: Maximum difference in TPR or FPR between groups
  - Ensures similar error rates across groups

- **Equal Opportunity**: Difference in true positive rates
  - Ensures equal benefit to qualified individuals

- **Disparate Impact**: Ratio of positive rates
  - Ratio < 0.8 often indicates bias (80% rule)

### Our Results:

In this demo, we created data with **intentional bias** where Group A has a 70% positive rate while Group B has only 40%. The framework successfully detected this bias!

## 8. Next Steps

Now that you've seen the basics, try:

1. **Load real data**: Use one of our case studies
   ```python
   df = pd.read_csv('../../data/case_studies/adult/adult.csv')
   ```

2. **Try different metrics**: Explore all available fairness metrics

3. **Experiment with thresholds**: Change the bias detection threshold
   ```python
   detector.set_threshold(0.05)  # Stricter threshold
   ```

4. **Multiple protected attributes**: Analyze intersectional fairness
   ```python
   detector.set_sensitive_attributes(['race', 'sex'])
   ```

5. **Check out other notebooks**:
   - `02_experiment_1.ipynb`: Full auto-detection experiment
   - `03_visualization.ipynb`: Advanced visualizations
   - `04_case_studies.ipynb`: Real-world examples

## 📚 Resources

- **Documentation**: See `../../docs/`
- **Paper**: `../../paper/main/main.pdf`
- **API Reference**: `../../docs/api/`
- **Examples**: `../../experiments/scripts/`

## 🎓 Congratulations!

You've completed the quick demo. You now know how to:
- ✓ Initialize the FairnessDetector
- ✓ Detect bias in datasets
- ✓ Interpret fairness metrics
- ✓ Visualize results

**Ready to detect bias in your ML models?** 🚀